# Compound Fitting with `SAMMY`

This Jupyter Notebook cell demonstrates the application of the `SAMMY` tool for fitting spectral transmission data using the `pleiades` utilities.

The transmission data utilized in this analysis originates from the [RPI measurement](https://doi.org/10.1016/j.anucene.2014.01.041) of enriched samples of [Eu-nat](https://www-nds.iaea.org/exfor/servlet/X4sGetSubent?subID=14382008) and [Eu-153](https://www-nds.iaea.org/exfor/servlet/X4sGetSubent?subID=14382007), which are documented on [EXFOR](https://www-nds.iaea.org/index-meeting-crp/CM-RF-2013/).

Europium comprises two stable isotopes, Eu-151 and Eu-153, with natural abundances of 47.8% and 52.2%, respectively. This example focuses on determining the abundances of these isotopes in both natural and enriched samples solely based on fitting the resonance transmission spectrum.

The analysis involves two main steps:

1. **Individual Isotope Fitting:** SAMMY is executed separately for each isotope. A simple configuration file is used to generate the resonance parameter file (.par file) employing resonance data directly sourced from ENDF8.

2. **Compound Isotopic Fitting:** The resonance spectra of both isotopes are combined to form an isotopic compound. SAMMY is then run again to fit the resonance spectrum and determine the most probable abundances of the isotopes.

In [1]:
%matplotlib inline
import pandas

# add pleiades to PYTHONPATH
import sys
sys.path.append("/sammy/Pleiades")

# add sammy to PATH
import os
os.environ["PATH"] += os.pathsep + "/sammy/sammy/build/bin"

## Configuration File for a Single Isotope

In this section, we provide an overview of the essential parameters within the configuration file. The configuration file is structured according to SAMMY conventions and contains vital information for the conversion process. Below, we focus on the key parameters relevant to this example:

- **ELMNT:** Specifies the element's name, denoted as 'Eu_151'. This identifier is crucial for SAMMY to recognize the isotope.

- **AW:** Is the isotopic mass. If `pleiades` identifies the `ELMNT` name, it automatically retrieves the mass from the AME database. Therefore, this parameter can be  set as 'auto'.

- **EMIN, EMAX:** Define the energy range for the analysis, indicating the minimum and maximum energies considered.

- **Commands:** This section includes essential commands for the configuration and generation of the `.par` file. Of particular importance is the 'INPUT IS ENDF/B FILE' command, instructing SAMMY to locate the reference to resonance data in the ENDF file. A MAT number will be replaced during subsequent steps, based on the isotope name.

- **Fit Parameters:** Defines crucial parameters for the fitting process:
  - `TEMP:` Represents the temperature in Kelvin.
  - `DIST:` Signifies the TOF (Time of Flight) distance in meters.
  - `CRFN:` Indicates the channel radius (not crucial at this stage).
  - `THICK:` Denotes the target thickness in units of atoms/barn.
  - `CROSS:` Specifies the reaction type, set here to 'TRANSMISSION.'

[comment]: # (- **Spin Groups:** This section is optional for this step. Here, two spin groups for the isotopes are specified. SAMMY automatically extracts this data from ENDF.)

Below is an example of the configuration file structure:

```ini
[Card1]
TITLE = input to convert ENDF8 file into a SAMMY par file

[Card2]
ELMNT = Eu-151
AW = auto
EMIN = 0.001
EMAX = 98.8

[Card3]
commands = CHI_SQUARED,CSISRS,NO_SOLVE_BAYES,USE_ENDF_ENERGY,INPUT IS ENDF/B FILE

[Card5]
TEMP = 296.
DIST = 25.584

[Card7]
CRFN = 1.0
THICK = 0.0021

[Card8]
CROSS = TRANSMISSION 


we save this file as `config_Eu_151.ini` and use `pleiades` to parse the file

In [2]:

from pleiades import sammyInput

# read config files and write inp files
saminp = sammyInput.InputFile("config_Eu_151.ini").process().write("Eu_151.inp")

# update the element name to Eu_153 and write a new file. 
# The matnumber and atomic weight are updates automaically
saminp.data["Card2"]["elmnt"] = "Eu-153"
saminp.data["Card2"]["aw"] = "auto"
saminp.process().write("Eu_153.inp")


In [3]:
!cat Eu_151.inp

input to convert ENDF8 file into a SAMMY par file                               
Eu-151      150.9199    0.0010   98.8000
CHI SQUARED IS WANTEd
CSISRS
DO NOT SOLVE BAYES EQUATIONS
QUANTUM NUMBERS ARE in parameter file
USE ENERGY RANGE FROm endf/b file
INPUT IS ENDF/B FILE MAT=6325

  296.0000   25.5840
    1.0000    0.0021
TRANSMISSION                                                                    



check the content of .inp files produced

In [4]:
!cat Eu_153.inp

input to convert ENDF8 file into a SAMMY par file                               
Eu-153      152.9212    0.0010   98.8000
CHI SQUARED IS WANTEd
CSISRS
DO NOT SOLVE BAYES EQUATIONS
QUANTUM NUMBERS ARE in parameter file
USE ENERGY RANGE FROm endf/b file
INPUT IS ENDF/B FILE MAT=6331

  296.0000   25.5840
    1.0000    0.0021
TRANSMISSION                                                                    



## Run SAMMY with ENDF data to produce .par file
For this step we use the `pleiades.sammyRunner` utility function `run_endf` which takes the inp files we have just created and run sammy to produce a par files using data from ENDF resonance tables.
The data is stored under separate directories at the `archive` directory

In [5]:
from pleiades import sammyRunner

sammyRunner.run_endf(inpfile="Eu_151.inp")
sammyRunner.run_endf(inpfile="Eu_153.inp")

Check that the par files were indeed created

In [6]:
ls archive/*/results

archive/Eu_151/results:
Eu_151.inp  Eu_151.lpt  Eu_151.par

archive/Eu_153/results:
Eu_153.inp  Eu_153.lpt  Eu_153.par


## Combine isotopic files into a compound
Now we want to take the two separate `.inp` and `.par` files that were created for each isotope of europium and create a compound that contains these two isotopes.

For this we will use the `pleiades.sammyParFile` utilities to create a `ParFile` object for each of the isotopes, combine it into one compound and write a new file that we can later run through `SAMMY`

For this example, we will set an equal weight (isotopic abundence) of 0.5 to each of the isotopes, and let `SAMMY` later deduce the correct weight according to fit over real data

In [7]:
from pleiades import sammyParFile

Eu_151 = sammyParFile.ParFile("archive/Eu_151/results/Eu_151.par",weight=0.5).read()
Eu_153 = sammyParFile.ParFile("archive/Eu_153/results/Eu_153.par",weight=0.5).read()

Now we form a compound by adding the two isotopes. `pleiades` will combine the resonance data from each isotope and create a new ParFile object that has all the neccasary data.

Then we use the `write` method to write the new `.par` into file.

In [8]:
Eu_nat = Eu_151 + Eu_153

Eu_nat.write("Eu_nat.par")

Let's check the content of this par file. 

Notice that the resonance parameters are all fixed, and that the abundances are set to **vary**. This is the default behaviour, later on, we will check what hapens when we vary different parameter, and check the effect of that change on the fit results

In [9]:
!cat Eu_nat.par

PARTICLE PAIR DEFINITIONS
Name=Eu_151       Particle a=neutron       Particle b=Other   
     Za= 0        Zb=63         Pent=1     Shift=0
     Sa=  0.5     Sb=   2.5     Ma=   1.008664915780000     Mb= 150.916445000000010
Name=Eu_153       Particle a=neutron       Particle b=Other   
     Za= 0        Zb=63         Pent=1     Shift=0
     Sa=  0.5     Sb=   2.5     Ma=   1.008664915780000     Mb= 152.921671000000003
 
SPIN GROUP INFORMATION
  1      1    0  2.0 0.5000000                                                  
    1  Eu_151      0         2            7.90000000 7.34863269                 
  2      1    0  3.0 0.5000000                                                  
    1  Eu_151      0         3            7.90000000 7.34863269                 
  3      1    0  2.0 0.5000000                                                  
    1  Eu_153      0         2            8.20000000 7.37750905                 
  4      1    0  3.0 0.5000000                                     

To run SAMMY on that new `.par` file we will need an `.inp` file as well. For that we can use the same `InpFile` object that we created for the `Eu_153` and just change the data a little bit

In [12]:
# update a title
saminp.data["Card1"]["title"] = "Run SAMMY to find abundence of europium isotopes"

# update compound name and assumed atomic weight
saminp.data["Card2"]["elmnt"] = "Eu_nat"
saminp.data["Card2"]["aw"] = 151.96

# update commands. We need to preform the REICH_MOORE and SOLVE_BAYES for this case
saminp.data["Card3"]["commands"] = 'CHI_SQUARED,CSISRS,SOLVE_BAYES,QUANTUM_NUMBERS,REICH_MOORE_FORM'

saminp.data["Card7"]["thick"] = 0.0021 # atoms/barn

# write the new input to file
# we set auto_update to False, since we don't want to replace "auto" values this time
saminp.process(auto_update=False).write("Eu_nat.inp")